In [1]:
from framework.utilities.metrics import calculate_bleu, calculate_rouge
import numpy as np

In [2]:
import pickle
import pandas as pd

with open('data/input_output_train', 'rb') as file:
    combined_outputs = pickle.load(file)

df_combined_outputs = pd.DataFrame(combined_outputs)
df_combined_outputs.head()

,wmt14,cnn_dailymail,gsm8k
0,{'input_text': 'Gutach: Noch mehr Sicherheit f...,{'input_text': '(CNN)The Palestinian Authority...,"{'input_text': None, '7b': None, 'tiny': None,..."
1,{'input_text': 'Sie stehen keine 100 Meter von...,{'input_text': '(CNN)Never mind cats having ni...,"{'input_text': None, '7b': None, 'tiny': None,..."
2,{'input_text': 'Zwei Anlagen so nah beieinande...,{'input_text': '(CNN)If you've been following ...,"{'input_text': None, '7b': None, 'tiny': None,..."
3,{'input_text': 'Diese Frage hat Gutachs Bürger...,{'input_text': '(CNN)Five Americans who were m...,"{'input_text': None, '7b': None, 'tiny': None,..."
4,"{'input_text': '""Die Rathausampel ist damals i...",{'input_text': '(CNN)A Duke student has admitt...,"{'input_text': None, '7b': None, 'tiny': None,..."


In [19]:
total_bleu_13b = 0
total_bleu_7b = 0
total_bleu_tiny = 0
num_rows = 0

for row in df_combined_outputs['wmt14']:
    num_rows += 1
    total_bleu_13b += calculate_bleu(row['13b'], row['label'])
    total_bleu_7b += calculate_bleu(row['7b'], row['label'])
    total_bleu_tiny += calculate_bleu(row['tiny'], row['label'])

In [20]:
avg_bleu_13b = total_bleu_13b / num_rows
avg_bleu_7b = total_bleu_7b / num_rows
avg_bleu_tiny = total_bleu_tiny / num_rows

print(f"Avg BLEU for 13B: {avg_bleu_13b}")
print(f"Avg BLEU for 7B: {avg_bleu_7b}")
print(f"Avg BLEU for Tiny: {avg_bleu_tiny}")

Avg BLEU for 13B: 0.5513241450960581
Avg BLEU for 7B: 0.5636676927707647
Avg BLEU for Tiny: 0.49071299218671666


In [21]:
h_tilde = .51

In [22]:
p_t_13b = h_tilde / .551
p_t_tiny = h_tilde / .491

In [23]:
print(p_t_13b)
print(p_t_tiny)

0.925589836660617
1.0386965376782078


In [24]:
p_t_13b = (h_tilde - avg_bleu_tiny) / (avg_bleu_13b - avg_bleu_tiny)
p_t_tiny = 1 - p_t_13b

In [25]:
print(p_t_13b)
print(p_t_tiny)

0.31820889205212305
0.6817911079478769


In [6]:
def sample_model(p_13b, p_tiny):
    return np.random.choice(['13b', 'tiny'], p=[p_13b, p_tiny])

In [27]:
total_sampled_bleu = 0
selected_model_counts = {'13b': 0, 'tiny': 0}

for row in df_combined_outputs['wmt14']:
    if row['13b'] and row['tiny'] and row['label']:  
        selected_model = sample_model(p_t_13b, p_t_tiny)
        selected_model_counts[selected_model] += 1

        bleu_score = calculate_bleu(row[selected_model], row['label'])
        total_sampled_bleu += bleu_score

avg_sampled_bleu = total_sampled_bleu / num_rows

print(f"Average BLEU score for selected models: {avg_sampled_bleu}")
print(f"Selected model counts: {selected_model_counts}")

Average BLEU score for selected models: 0.5085141289424104
Selected model counts: {'13b': 938, 'tiny': 2060}


In [28]:
(938 * 527.870)/3000

165.04735333333332

In [29]:
(2060 * 44.639)/3000

30.652113333333336

In [4]:
total_rouge_13b = 0
total_rouge_7b = 0
total_rouge_tiny = 0
num_rows = 0

for row in df_combined_outputs['cnn_dailymail']:
    num_rows += 1
    total_rouge_13b += calculate_rouge(row['13b'], row['label'])['rouge1']
    total_rouge_7b += calculate_rouge(row['7b'], row['label'])['rouge1']
    total_rouge_tiny += calculate_rouge(row['tiny'], row['label'])['rouge1']

In [5]:
avg_rouge_13b = total_rouge_13b / num_rows
avg_rouge_7b = total_rouge_7b / num_rows
avg_rouge_tiny = total_rouge_tiny / num_rows

print(f"Avg ROUGE for 13B: {avg_rouge_13b}")
print(f"Avg ROUGE for 7B: {avg_rouge_7b}")
print(f"Avg ROUGE for Tiny: {avg_rouge_tiny}")

Avg ROUGE for 13B: 0.32213102025411433
Avg ROUGE for 7B: 0.32457467037783444
Avg ROUGE for Tiny: 0.30924118236882664


In [33]:
h_tilde = .3125

In [29]:
p_t_13b = (h_tilde - avg_rouge_tiny) / (avg_rouge_13b - avg_rouge_tiny)
p_t_tiny = 1 - p_t_13b
print(p_t_13b)
print(p_t_tiny)

0.25282068402838
0.74717931597162


In [30]:
total_sampled_rouge = 0
selected_model_counts = {'13b': 0, 'tiny': 0}

for row in df_combined_outputs['cnn_dailymail']:
    if row['13b'] and row['tiny'] and row['label']:  
        selected_model = sample_model(p_t_13b, p_t_tiny)
        selected_model_counts[selected_model] += 1

        rouge_score = calculate_rouge(row[selected_model], row['label'])
        total_sampled_rouge += rouge_score['rouge1']

avg_sampled_rouge = total_sampled_rouge / num_rows

print(f"Average ROUGE score for selected models: {avg_sampled_rouge}")
print(f"Selected model counts: {selected_model_counts}")

Average ROUGE score for selected models: 0.3084090454829778
Selected model counts: {'13b': 739, 'tiny': 2220}


In [31]:
(739 * 750.285)/3000

184.820205

In [32]:
(2220 * 142.08)/3000

105.13920000000002